In [13]:
import importlib
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable

import mysklearn.myruleminer
importlib.reload(mysklearn.myruleminer)
from mysklearn.myruleminer import MyAssociationRuleMiner

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import notebookutils
importlib.reload(notebookutils)
import notebookutils as nbutl

# Classification Results

In [14]:
league_standing_pytable = MyPyTable()
league_standing_pytable.load_from_file("lib/nhl_leaguestandings.csv")

league_standing_pytable.remove_rows_with_missing_values()

league_standing_pytable.drop_col("ROW")
league_standing_pytable.drop_col("RPt%")
league_standing_pytable.drop_col("OL")
league_standing_pytable.drop_col("T")
league_standing_pytable.drop_col("SEASON_TM")
league_standing_pytable.drop_col("SEASON")
league_standing_pytable.drop_col("CONFERENCE")
league_standing_pytable.drop_col("DIVISION")
league_standing_pytable.drop_col("TEAM")
league_standing_pytable.drop_col("TM")
league_standing_pytable.drop_col("PLAYOFFS")



#nbutl.discretize_ptspercent(league_standing_pytable.data, )

#'GP', 'W', 'L','PTS','PTS%', 'GF', 'SRS'

key = league_standing_pytable.get_key(league_standing_pytable.column_names, ['GP', 'W', 'L','PTS','PTS%', 'GF', 'GA', 'SRS', 'SOS'])
nbutl.discretize_gf(league_standing_pytable.data, key[0])
nbutl.discretize_gf(league_standing_pytable.data, key[1])
nbutl.discretize_gf(league_standing_pytable.data, key[2])
nbutl.discretize_gf(league_standing_pytable.data, key[3])
nbutl.discretize_ptspercent(league_standing_pytable.data, key[4])
nbutl.discretize_srs(league_standing_pytable.data, key[7])
nbutl.discretize_sos(league_standing_pytable.data, key[8])
nbutl.discretize_gf(league_standing_pytable.data, key[5])
nbutl.discretize_gf(league_standing_pytable.data, key[6])


y_col_name = "FINISH"

y = league_standing_pytable.get_column(y_col_name)
league_standing_pytable.drop_col(y_col_name)
X = [league_standing_pytable.data[i] for i in range(len(league_standing_pytable.data))]


X_train_folds, X_test_folds  = myevaluation.stratified_kfold_cross_validation(X, y, 10)

In [15]:
from tabulate import tabulate

# descision tree

#['GP','W','L','PTS','PTS%','GF','GA','SOS']

print("===========================================")
print("DESCISION TREE")
print("===========================================")

true_tr_strat, pred_tr_strat = notebookutils.kfoldstrat(league_standing_pytable, X, y, "tree",['GP','W','L','PTS','PTS%','GF','GA','SRS','SOS'])

tr_accuracy_strat = myevaluation.accuracy_score(true_tr_strat, pred_tr_strat)

print("Descision Tree Classifier")
print("     Accuracy = " + str(round(tr_accuracy_strat,3)) + ", Error Rate = " + str(round(1-tr_accuracy_strat,3)))
print("     Precision = " + str(round(myevaluation.binary_precision_score(true_tr_strat,pred_tr_strat),3)))
print("     Recall = " + str(round(myevaluation.binary_recall_score(true_tr_strat,pred_tr_strat),3)))
print("     F1 Score = " + str(round(myevaluation.binary_f1_score(true_tr_strat,pred_tr_strat),3)))
print("     Confusion Matrix\n" + tabulate(notebookutils.matrix(true_tr_strat, pred_tr_strat), headers=["Outcome","0","1", "2", "3", "4", "5", "Total", "Recognition(%)"]))


DESCISION TREE
Descision Tree Classifier
     Accuracy = 0.496, Error Rate = 0.504
     Precision = 0.346
     Recall = 0.463
     F1 Score = 0.396
     Confusion Matrix
  Outcome    0    1    2    3    4    5    Total    Recognition(%)
---------  ---  ---  ---  ---  ---  ---  -------  ----------------
        0  429   74    2    2    0    1      508              84.4
        1   86  223    2   11    0    2      324              68.8
        2   46  146    1    9    0    4      206               0.5
        3   41   98    4   10    0    3      156               6.4
        4   13   54    0    8    0    3       78               0
        5    8   49    4    9    0    8       78              10.3


In [16]:
print("===========================================")
print("NAIVE BAYES")
print("===========================================")


true_nb_strat, pred_nb_strat = notebookutils.kfoldstrat(league_standing_pytable, X, y, "naive",['GP','W','L','PTS','PTS%','GF','GA','SRS','SOS'])


nb_accuracy_strat = myevaluation.accuracy_score(true_nb_strat, pred_nb_strat)


print("Naive Bayes Classifier")
print("     Accuracy = " + str(round(nb_accuracy_strat,3)) + ", Error Rate = " + str(round(1-nb_accuracy_strat,3)))
print("     Precision = " + str(round(myevaluation.binary_precision_score(true_nb_strat,pred_nb_strat),3)))
print("     Recall = " + str(round(myevaluation.binary_recall_score(true_nb_strat,pred_nb_strat),3)))
print("     F1 Score = " + str(round(myevaluation.binary_f1_score(true_nb_strat,pred_nb_strat),3)))
print("     Confusion Matrix\n" + tabulate(notebookutils.matrix(true_nb_strat, pred_nb_strat), headers=["Outcome","0","1", "2", "3", "4", "5", "Total", "Recognition(%)"]))

NAIVE BAYES
Naive Bayes Classifier
     Accuracy = 0.521, Error Rate = 0.479
     Precision = 0.41
     Recall = 0.313
     F1 Score = 0.355
     Confusion Matrix
  Outcome    0    1    2    3    4    5    Total    Recognition(%)
---------  ---  ---  ---  ---  ---  ---  -------  ----------------
        0  424   73    4    3    3    1      508              83.5
        1   63  178   38   15    5   25      324              54.9
        2   33  101   33    9    5   25      206              16
        3   28   45   16   22   15   30      156              14.1
        4    9   20    8   11   16   14       78              20.5
        5    1   17   11   17    2   30       78              38.5


In [17]:
print("===========================================")
print("kNN CLASSIFIER")
print("===========================================")


true_knn_strat, pred_knn_strat = notebookutils.kfoldstrat(league_standing_pytable, X, y, "knn",['GP','W','L','PTS','PTS%','GF','GA','SRS','SOS'])

knn_accuracy_strat = myevaluation.accuracy_score(true_knn_strat, pred_knn_strat)


print("k Nearest Neighbors Classifier")
print("     Accuracy = " + str(round(knn_accuracy_strat,3)) + ", Error Rate = " + str(round(1-knn_accuracy_strat,3)))
print("     Precision = " + str(round(myevaluation.binary_precision_score(true_knn_strat,pred_knn_strat),3)))
print("     Recall = " + str(round(myevaluation.binary_recall_score(true_knn_strat,pred_knn_strat),3)))
print("     F1 Score = " + str(round(myevaluation.binary_f1_score(true_knn_strat,pred_knn_strat),3)))
print("     Confusion Matrix\n" + tabulate(notebookutils.matrix(true_knn_strat, pred_knn_strat), headers=["Outcome","0","1", "2", "3", "4", "5", "Total", "Recognition(%)"]))
print("")

kNN CLASSIFIER
k Nearest Neighbors Classifier
     Accuracy = 0.243, Error Rate = 0.757
     Precision = 0.239
     Recall = 0.988
     F1 Score = 0.385
     Confusion Matrix
  Outcome    0    1    2    3    4    5    Total    Recognition(%)
---------  ---  ---  ---  ---  ---  ---  -------  ----------------
        0    8  500    0    0    0    0      508               1.6
        1    3  320    1    0    0    0      324              98.8
        2    0  206    0    0    0    0      206               0
        3    0  156    0    0    0    0      156               0
        4    0   78    0    0    0    0       78               0
        5    0   78    0    0    0    0       78               0



In [18]:
print("===========================================")
print("Rule Miner")
print("===========================================")

rule_miner = MyAssociationRuleMiner(minsup=0.05, minconf=.2)
rule_miner.fit(league_standing_pytable.data)
rule_miner.print_association_rules()



Rule Miner
Rule Table
  #  RULE                 SUPPORT    CONFIDENCE      LIFT
---  -----------------  ---------  ------------  --------
  1  IF 4 THEN 5        0.0644444      0.517857  1.60346
  2  IF 4 THEN 7        0.101481       0.815476  1.01092
  3  IF 6 THEN 5        0.211852       0.276329  0.855604
  4  IF 5 THEN 6        0.211852       0.655963  0.855604
  5  IF 7 THEN 5        0.245926       0.304867  0.943968
  6  IF 5 THEN 7        0.245926       0.761468  0.943968
  7  IF 7 THEN 6        0.573333       0.710744  0.927057
  8  IF 6 THEN 7        0.573333       0.747826  0.927057
  9  IF 8 THEN 6        0.131111       0.783186  1.02155
 10  IF 8 THEN 7        0.121481       0.725664  0.899583
 11  IF 5 AND 7 THEN 4  0.0548148      0.222892  1.79109
 12  IF 4 AND 7 THEN 5  0.0548148      0.540146  1.67247
 13  IF 4 AND 5 THEN 7  0.0548148      0.850575  1.05443
 14  IF 4 THEN 5 AND 7  0.0548148      0.440476  1.79109
 15  IF 6 AND 7 THEN 5  0.134815       0.235142  0.728078

In [19]:
print("===========================================")
print("RANDOM FOREST CLASSIFIER")
print("===========================================")


true_rand_forest_strat, pred_rand_forest_strat = notebookutils.kfoldstrat(league_standing_pytable, X, y, "rand_forest",['GP','W','L','PTS','PTS%','GF','GA','SRS','SOS'])

rand_forest_accuracy_strat = myevaluation.accuracy_score(true_rand_forest_strat, pred_rand_forest_strat)


print("Random Forest Classifier")
print("     Accuracy = " + str(round(rand_forest_accuracy_strat,3)) + ", Error Rate = " + str(round(1-rand_forest_accuracy_strat,3)))
print("     Precision = " + str(round(myevaluation.binary_precision_score(true_rand_forest_strat,pred_rand_forest_strat),3)))
print("     Recall = " + str(round(myevaluation.binary_recall_score(true_rand_forest_strat,pred_rand_forest_strat),3)))
print("     F1 Score = " + str(round(myevaluation.binary_f1_score(true_rand_forest_strat,pred_rand_forest_strat),3)))
print("     Confusion Matrix\n" + tabulate(notebookutils.matrix(true_rand_forest_strat, pred_rand_forest_strat), headers=["Outcome","0","1", "2", "3", "4", "5", "Total", "Recognition(%)"]))
print("")

RANDOM FOREST CLASSIFIER
Random Forest Classifier
     Accuracy = 0.5, Error Rate = 0.5
     Precision = 0.344
     Recall = 0.443
     F1 Score = 0.387
     Confusion Matrix
  Outcome    0    1    2    3    4    5    Total    Recognition(%)
---------  ---  ---  ---  ---  ---  ---  -------  ----------------
        0  433   69    4    0    1    1      508              85.2
        1   88  213   15    7    0    1      324              65.7
        2   43  137   15   11    0    0      206               7.3
        3   30   98   13   11    2    2      156               7.1
        4   10   53    7    6    1    1       78               1.3
        5    5   50   11    9    0    3       78               3.8

